# Import

In [1]:
# importing
import numpy as np
import scipy.signal as signal
import scipy.stats as stats
import scipy as sp

import matplotlib.pyplot as plt
import matplotlib

from matplotlib import animation, rc
from matplotlib.animation import PillowWriter # Disable if you don't want to save any GIFs.

from ipywidgets import interactive
import ipywidgets as widgets

# showing figures inline
%matplotlib inline

In [2]:
# plotting options 
font = {'size'   : 30}
plt.rc('font', **font)
plt.rc('text', usetex=True)

matplotlib.rc('figure', figsize=(24, 12) )

# Here we go

In [3]:
# define our own "histogram"
def get_quantization( samples, width,  numb_of_bins ):
    '''
    determines intervals and hits within the intervals when
    being provided with samples, width of region, and number of bins

    NOTE: provides (integer) number of hits, not normalized to density 
    '''

    # get number of samples and Delta (in x)
    n = len( samples )
    Delta = width / numb_of_bins

    # first define no. of hits per region
    hits_regions = np.zeros( numb_of_bins )
    
    # get interval points
    interval_points = np.arange( 1, numb_of_bins )* Delta - width / 2 

    # walk through all samples and assign them to one quant. level
    for s in samples:
        
        # find index for mapping s 
        try:
            index = np.min( np.where( s < interval_points ) )
            hits_regions[ index ] += 1
        except:
            hits_regions[ -1 ] += 1

    return interval_points, hits_regions

ip, vals = get_quantization( np.random.randn(10), 1, 6)
print( ip )
print( vals)


[-0.33333333 -0.16666667  0.          0.16666667  0.33333333]
[2. 0. 1. 2. 1. 4.]


In [4]:
# perform chi square test
def do_chi_square( h, interval_points, alpha  ):
    '''
    determines chi square test
    thereby determining theoretical probabiliy of regions provided as arguments

    NOTE: expects (integer) number of hits, not normalized to density 
    '''
    
    # get number of samples and numb_of_bins
    n = np.sum( h )
    numb_of_bins = len( interval_points ) + 1
    
    # init prob. vector and determine theoretical prob.
    interval_points_extended = np.hstack( ( -np.inf, interval_points, np.inf ) )

    p = np.zeros( len( interval_points_extended ) - 1 )
    for _n in range( numb_of_bins ):
        p[ _n ] = stats.norm.cdf( interval_points_extended[ _n + 1 ] ) - stats.norm.cdf( interval_points_extended[ _n ] )

        
    # get empirical test value for chi square test
    t = np.sum( ( h - n * p )**2 / ( n *p  ) )

    # get theoretical test value for chi square test
    chi2 = stats.chi2.ppf( 1 - alpha, numb_of_bins - 1 )

    
    return t, chi2


# ip, vals = get_quantization( np.random.randn(100), 10, 20 )
# t, chi2 = do_chi_square( vals, ip, .1 )

In [5]:
# interactive plotting function
def show_test_result( n, alpha, width, numb_of_bins ):
    '''
    showing result of chi square test for samples generated according to given variance

    test is performed by binning x axis as numb_of_bins bins in [a,b]
    --> show effects of "not being gaussian" for "stupid" partition of real axis
    '''


    # get samples
    samples = np.random.randn( n  )


    # get our own quant. and map to histogram
    ip, hits = get_quantization( samples, width, numb_of_bins )

    hits = np.ones( numb_of_bins )

    # map interval points to their midpoints to match length of hist
    if len( ip ) ==3:
            midpoints = np.array( [ ip[ 1 ] - width / 2, ip[ 1 ] + width / 2 ] ) 

    elif len( ip ) > 3:
            dist = ip[ 2 ] - ip[ 1 ]
            
            midpoints = ip - dist / 2
            midpoints = np.hstack( ( midpoints, midpoints[0] ) )

    # hist shoulkd be normalized to density=True
    my_hist = hits / n * numb_of_bins / width



    # now do chi square test
    t, chi2 = do_chi_square( hits, ip, alpha )
    print( 'Test statistic t:\t{}'.format( t ) )
    print( 'Chi-square value:\t{}'.format( chi2 ) )

    test_result = 'H_0 declined' if t > chi2 else 'H_0 not declined'

    print( 'Niveau: \t\t{}'.format( alpha) ) 
    print( 'Test result: \t\t' + test_result + ' with error prob. {}'.format( alpha) )



    # figure for histogram and theory
    plt.figure( figsize=( 20, 8 ) )
    font = {'size'   : 14}
    plt.rc('font', **font)
    plt.rc('text', usetex=matplotlib.checkdep_usetex(True))


    # plot histogram of numpy
    plt.hist( samples, bins = 50, density=True, label='histogram of numpy')

    # plot theoretical gaussian
    x = np.linspace( -10, 10, 1000 )
    plt.plot( x, 1/np.sqrt(2*np.pi) * np.exp( - x**2 / 2 ), color='red', label='theory' )

    # plot "our" histogram
    plt.bar( midpoints, my_hist, color='green', width=width/numb_of_bins )


    # some nice little add-ons
    plt.xlabel(r'$k$',fontsize=14)
    plt.ylabel(r'$H_n(k)$',fontsize=14)
    plt.legend()
    plt.grid( True )


interactive_update = interactive( 
    show_test_result, 
    n = widgets.IntSlider(
            min = 10, max = 10000, step = 1, value = 500, 
            continuous_update = False, 
            description = 'observation length n', 
            style={'description_width': 'initial'}, 
            layout=widgets.Layout(width='50%'),
            align_items='center',
            ),
    alpha = widgets.FloatSlider(
            min = 0.0, max = 0.5, step = 0.01, value = 0.1, 
            continuous_update = False, 
            description = 'alpha', 
            style={'description_width': 'initial'}, 
            layout=widgets.Layout(width='50%'),
            align_items='center',
            ),
    width = widgets.FloatSlider(
            min = 0.0, max = 10, step = 0.1, value = 1.0, 
            continuous_update = False, 
            description = 'quantization width for test', 
            style={'description_width': 'initial'}, 
            layout=widgets.Layout(width='50%'),
            align_items='center',
            ),
    numb_of_bins = widgets.IntSlider(
            min = 10, max = 100, step = 1, value = 1, 
            continuous_update = False, 
            description = 'no. bins for test', 
            style={'description_width': 'initial'}, 
            layout=widgets.Layout(width='50%'),
            align_items='center',
            )
    )


output = interactive_update.children[-1]
output.layout.height = '500px'
interactive_update

interactive(children=(IntSlider(value=500, continuous_update=False, description='observation length n', layout…

# Now there should be a sequence of multiple samplings showing that the error rate of the test is actually equal to alpha